<a href="https://colab.research.google.com/github/ArturHugo/PLN-2022-1/blob/main/Atividade07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 07 - CNN

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, ReLU, BatchNormalization, Dense, Flatten, Dropout, Resizing
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
import os

In [2]:
model = Sequential()
model.add(Resizing(224, 224,
                   interpolation="bilinear",
                   input_shape=(256,256,3)))
model.add(Conv2D(filters=96,
                 kernel_size=11,
                 strides=(4, 4),
                 activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
                       strides=(2,2)))
model.add(Conv2D(filters=256,
                 kernel_size=5,
                 strides=(1, 1),
                 padding="same",
                 activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
                       strides=(2,2)))
model.add(Conv2D(filters=384,
                 kernel_size=3,
                 strides=(1, 1),
                 padding="same",
                 activation="relu"))
model.add(Conv2D(filters=384,
                 kernel_size=3,
                 strides=(1, 1),
                 padding="same",
                 activation="relu"))
model.add(Conv2D(filters=256,
                 kernel_size=3,
                 strides=(1, 1),
                 padding="same",
                 activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),
                       strides=(2,2)))
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 54, 54, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1

In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [5]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/PLN/full_3.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/Colab Notebooks/PLN/")

In [6]:
path_to_images = "/content/drive/MyDrive/Colab Notebooks/PLN/full_3/"
image_paths_list = []
image_labels_list = []
for dir in os.listdir(path_to_images):
  for filename in os.listdir(path_to_images + '/' + dir):
    image_labels_list.append(dir)
    image_paths_list.append(path_to_images + '/' + dir + '/' + filename)

In [7]:
df = pd.DataFrame(columns=['image_path', 'label'])
df['image_path'] = image_paths_list
df['label'] = image_labels_list
df = df.sample(frac=1)
df

,image_path,label
579,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,scientific _publication
1064,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,advertisement
2664,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,email
2716,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,email
1376,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,advertisement
...,...,...
814,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,scientific _publication
1615,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,advertisement
757,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,scientific _publication
1733,/content/drive/MyDrive/Colab Notebooks/PLN/ful...,advertisement


In [8]:
X = df['image_path'].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [9]:
one_hot_encoder = OneHotEncoder()

y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_valid_one_hot = one_hot_encoder.fit_transform(y_valid.reshape(-1, 1)).toarray()
y_test_one_hot = one_hot_encoder.fit_transform(y_test.reshape(-1, 1)).toarray()


label_enconder = LabelEncoder()

y_train_int = label_enconder.fit_transform(y_train)
y_valid_int = label_enconder.fit_transform(y_valid)
y_test_int = label_enconder.fit_transform(y_test)

In [10]:
df_train = pd.DataFrame(columns=['image_path', 'label'])
df_train['image_path'] = X_train
df_train['label'] = y_train

df_valid = pd.DataFrame(columns=['image_path', 'label'])
df_valid['image_path'] = X_valid
df_valid['label'] = y_valid

df_test = pd.DataFrame(columns=['image_path', 'label'])
df_test['image_path'] = X_test
df_test['label'] = y_test

In [11]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(df_train,
                                              directory_path=path_to_images,
                                              x_col='image_path',
                                              y_col='label',
                                              class_mode='categorical',
                                              shuffle=True)
valid_generator = datagen.flow_from_dataframe(df_valid,
                                              directory_path=path_to_images,
                                              x_col='image_path',
                                              y_col='label',
                                              class_mode='categorical',
                                              shuffle=True)
test_generator = datagen.flow_from_dataframe(df_test,
                                             directory_path=path_to_images,
                                             x_col='image_path',
                                             y_col='label',
                                             class_mode='categorical',
                                             shuffle=True)

Found 2072 validated image filenames belonging to 3 classes.
Found 444 validated image filenames belonging to 3 classes.
Found 445 validated image filenames belonging to 3 classes.


In [12]:
valid_generator.image_shape

(256, 256, 3)

In [ ]:
sgd = SGD(learning_rate=0.0001)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator,
          epochs=70,
          batch_size=16,
          validation_data=valid_generator)

In [19]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(256,256,3),
                   pooling='avg',classes=3,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(1024, activation='relu'))
resnet_model.add(Dense(3, activation='softmax'))
resnet_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_8 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 25,688,963
Trainable params: 2,101,251
Non-trainable params: 23,587,712
_________________________________________________________________


In [20]:
resnet_model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

resnet_model.fit(train_generator,
          epochs=70,
          batch_size=16,
          validation_data=valid_generator)

Epoch 1/70
65/65 [==============================] - 18s 230ms/step - loss: 1.1276 - accuracy: 0.3219 - val_loss: 1.0382 - val_accuracy: 0.3423
Epoch 2/70
65/65 [==============================] - 14s 221ms/step - loss: 1.0104 - accuracy: 0.4995 - val_loss: 0.9857 - val_accuracy: 0.6194
Epoch 3/70
65/65 [==============================] - 14s 216ms/step - loss: 0.9684 - accuracy: 0.6274 - val_loss: 0.9508 - val_accuracy: 0.6441
Epoch 4/70
65/65 [==============================] - 14s 216ms/step - loss: 0.9374 - accuracy: 0.6511 - val_loss: 0.9242 - val_accuracy: 0.6644
Epoch 5/70
65/65 [==============================] - 15s 226ms/step - loss: 0.9127 - accuracy: 0.6892 - val_loss: 0.9027 - val_accuracy: 0.6892
Epoch 6/70
65/65 [==============================] - 14s 218ms/step - loss: 0.8920 - accuracy: 0.7114 - val_loss: 0.8842 - val_accuracy: 0.7342
Epoch 7/70
65/65 [==============================] - 14s 216ms/step - loss: 0.8738 - accuracy: 0.7379 - val_loss: 0.8695 - val_accuracy: 0.7883

In [21]:
alexnet_results = model.evaluate(test_generator)
print(f"AlexNet results: {alexnet_results}")
resnet_results = resnet_model.evaluate(test_generator)
print(f"ResNet results: {resnet_results}")

14/14 [==============================] - 2s 161ms/step - loss: 0.2919 - accuracy: 0.9169
AlexNet results: [0.2918761074542999, 0.9168539047241211]
14/14 [==============================] - 3s 171ms/step - loss: 0.5527 - accuracy: 0.8472
ResNet results: [0.5527153611183167, 0.8471910357475281]
